In [ ]:
import mediapipe as mp
import os
import sys

# Import the main class from the provided code
# Assuming the provided code is saved as face_tracker.py
from Main_model.showframeVisualization import FrameShow_head_face
import cv2
import time
import numpy as np

def main():
    # Initialize the face tracker
    face_tracker = FrameShow_head_face(
        model_path='Main_model/face_landmarker.task',
        isVideo=False,  # Set to False for static image processing
        isHeadposeOn=False,
        isFaceOn=True
    )
    
    # # Enable visualization features
    face_tracker.set_labet_face_element(False)
    # face_tracker.set_IsShowBox(True)
    # face_tracker.set_IsShowHeadpose(True)
    
    # Create window
    window_name = "Face Tracking Demo"
    cv2.namedWindow(window_name, cv2.WINDOW_NORMAL)
    
    # Image path (fixed from your example)
    # image_path = '/Users/porchportal2/Desktop/Screenshot 2568-03-17 at 12.33.54.png'
    image_path = '/Users/porchportal2/Desktop/🔥everything/eyeTrackingProject/Program/📌Face_Head/special_dataset/webcam_136.jpg'
    
    # Load the image properly
    frame = cv2.imread(image_path)
    
    if frame is None:
        print(f"Error: Could not read image from {image_path}")
        return
    
    # Process the frame (timestamp_ms is only relevant for video mode)
    timestamp_ms = 0
    metrics, processed_frame = face_tracker.process_frame(frame, timestamp_ms, isVideo=False, isEnhanceFace=False)
    
    if metrics is None:
        print("No face detected in the image.")
    else:
        # Print face analysis results
        pitch, yaw, roll = metrics.head_pose_angles
        print(f"Head Pose - Pitch: {pitch:.2f}, Yaw: {yaw:.2f}, Roll: {roll:.2f}")
        print(metrics)
    
    # Display the processed image
    cv2.imshow(window_name, processed_frame)
    cv2.waitKey(0)  # Wait for a key press
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()

I0000 00:00:1745825277.063244 3713459 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.4), renderer: Apple M1 Max
W0000 00:00:1745825277.063624 3713459 face_landmarker_graph.cc:174] Sets FaceBlendshapesGraph acceleration to xnnpack by default.
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1745825277.067549 3713757 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1745825277.075643 3713757 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
I0000 00:00:1745825277.138228 3713459 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.4), renderer: Apple M1 Max
W0000 00:00:1745825277.138483 3713459 face_landmarker_graph.cc:174] Sets FaceBlendshapesGraph acceleration to xnnpack by default.
W0000 00:00:1745825277.142136 3713773 inference_feedback_manager.cc:114] Feedback manage

Using device: mps
Head Pose - Pitch: -0.19, Yaw: -0.25, Roll: 1.09
all_data_output(head_pose_angles=(-0.192516749688779, -0.25367589477705477, 1.0939493273312475), face_box=((774, 519), (1083, 902)), left_eye_box=((950, 639), (1022, 669)), right_eye_box=((836, 640), (904, 666)), eye_iris_center=((983, 653), (869, 652)), eye_iris_left_box=((970, 641), (996, 665)), eye_iris_right_box=((857, 641), (882, 665)), eye_centers=((986, 656), (869, 655), (927, 655)), landmark_positions={'Nose': (929, 716), 'Mouth_L': (885, 791), 'Mouth_R': (982, 787), 'Chin': (936, 877), 'Cheek_L': (801, 703), 'Cheek_R': (1058, 698)}, left_eye_state=('Open', np.float64(0.2776273191632896)), right_eye_state=('Open', np.float64(0.27117300405376954)), depths=(np.float64(70.32), np.float64(71.12), np.float64(72.05), np.float64(71.37)))


In [1]:
import os
import sys
import logging

# Suppress all warnings and low-level logs
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # Suppress TensorFlow warnings
os.environ['MEDIAPIPE_DISABLE_GPU'] = '1'  # Disable GPU warnings if not using GPU
os.environ['PYTHONWARNINGS'] = 'ignore'  # Suppress Python warnings

# Configure logging to suppress non-critical warnings
logging.basicConfig(level=logging.ERROR)

# Import other libraries (after setting environment variables)
import cv2
import time
import numpy as np
import torch

# Initialize absl logging before importing MediaPipe-related modules
try:
    from absl import logging as absl_logging
    absl_logging.set_verbosity(absl_logging.ERROR)
    # Additional absl flags configuration to minimize warnings
    from absl import flags
    flags.FLAGS.mark_as_parsed()
except ImportError:
    print("Warning: absl module not found, some warnings may still appear")

# Now import your custom module
from Main_model.showframeVisualization import FrameShow_head_face

class Toggle:
    def __init__(self, initial_state=False):
        self.state = initial_state
    
    def toggle(self):
        self.state = not self.state
        return self.state
    
    def __bool__(self):
        return self.state

def main():
    try:
        cap = cv2.VideoCapture(1)
        if not cap.isOpened():
            print("Error: Could not open camera. Trying default camera (0)...")
            cap = cv2.VideoCapture(0)
            if not cap.isOpened():
                print("Error: No camera available. Exiting.")
                return
        
        cap.set(cv2.CAP_PROP_FPS, 30)
        min_width = 1280
        min_height = 720

        # Get the maximum resolution supported by the camera
        max_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        max_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

        cap.set(cv2.CAP_PROP_FRAME_WIDTH, max(max_width, min_width))
        cap.set(cv2.CAP_PROP_FRAME_HEIGHT, max(max_height, min_height))
        
        start_time = time.time()
        isShow_direction = Toggle(False)
        isShow_label = Toggle(False)
        isShow_box = Toggle(False)
        isShow_mask = Toggle(False)
        IsValueOutput = Toggle(False)
        isZoomFace = Toggle(False)
        
        # Create tracker with error handling
        try:
            tracker = FrameShow_head_face(
                model_path='Main_model/face_landmarker.task', 
                isHeadposeOn=True, 
                isVideo=True,
                isFaceOn=True 
            )
        except Exception as e:
            print(f"Error initializing face tracker: {e}")
            print("Make sure 'face_landmarker.task' file exists in the current directory")
            return
        
        print("Face tracking started. Press 'q' to quit.")
        print("Controls:")
        print("  s: Toggle face element labels")
        print("  d: Toggle bounding box")
        print("  f: Toggle head pose visualization")
        print("  g: Toggle face mask")
        print("  e: Toggle value output")
        print("  z: Toggle face zoom")
        
        while True:
            ret, frame = cap.read()
            if not ret:
                print("Error: Failed to grab frame from camera")
                print("Try reconnecting your camera or checking permissions")
                break
            
            h, w = frame.shape[:2]
            x_axis = int(w // 1.5)
            current_time = time.time()
            timestamp_ms = int((current_time - start_time) * 1000)
            
            try:
                # Process frame with current timestamp
                matrix_out, frame = tracker.process_frame(frame, timestamp_ms)
                
                # Check if face was detected
                face_detected = matrix_out is not None
                status_color = (0, 255, 0) if face_detected else (0, 0, 255)
                
                # Add face detection status
                cv2.putText(frame, f"Face detected: {face_detected}", 
                           (x_axis, 80), cv2.FONT_HERSHEY_SIMPLEX, 0.9, status_color, 2)
                
                # Add controls info
                cv2.putText(frame, f"(s) show all label parameters: {bool(isShow_label)}", 
                           (x_axis, 120), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
                cv2.putText(frame, f"(d) show bounding box face: {bool(isShow_box)}", 
                           (x_axis, 160), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
                cv2.putText(frame, f"(f) show visualization headpose: {bool(isShow_direction)}", 
                           (x_axis, 200), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
                cv2.putText(frame, f"(g) show mask on face: {bool(isShow_mask)}", 
                           (x_axis, 240), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
                cv2.putText(frame, f"(e) Value Output: {bool(IsValueOutput)}", 
                           (x_axis, 280), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
                cv2.putText(frame, f"(z) Zoom: {bool(isZoomFace)}", 
                           (x_axis, 320), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
                
                # Make sure frame is properly formatted
                if isinstance(frame, list):  # If it's a list, convert to NumPy array
                    frame = np.array(frame, dtype=np.uint8)
                
                # Display the frame
                if frame is not None and frame.size > 0:
                    cv2.imshow('Head Pose Estimation', frame)
                else:
                    print("Error: Invalid frame!")
                
                # Only output values if face is detected AND value output is enabled
                if bool(IsValueOutput) and face_detected:
                    print("-" * 50)
                    print(f"Eye Left: {matrix_out.eye_centers[0]}, Right: {matrix_out.eye_centers[1]}")
                    print(f"Box Eye Left: Min({matrix_out.left_eye_box[0]}), Max({matrix_out.left_eye_box[1]})")
                    print(f"Box Eye Right: Min({matrix_out.right_eye_box[0]}), Max({matrix_out.right_eye_box[1]})")
                    print(f"Head-pose: Pitch{matrix_out.head_pose_angles[0]:.2f}, Yaw{matrix_out.head_pose_angles[1]:.2f}, Roll{matrix_out.head_pose_angles[2]:.2f}")
                elif bool(IsValueOutput) and not face_detected:
                    print("No face detected - cannot display metrics")
                    
                key = cv2.waitKey(1) & 0xFF
                if key == ord('q'):
                    break
                elif key == ord('s'):
                    isShow_label.toggle()
                    tracker.set_labet_face_element(bool(isShow_label))
                elif key == ord('d'):
                    isShow_box.toggle()
                    tracker.set_IsShowBox(bool(isShow_box))
                elif key == ord('f'):
                    isShow_direction.toggle()
                    tracker.set_IsShowHeadpose(bool(isShow_direction))
                elif key == ord('g'):
                    isShow_mask.toggle()
                    tracker.set_IsMaskOn(bool(isShow_mask))
                elif key == ord('e'):
                    IsValueOutput.toggle()
                elif key == ord('z'):
                    isZoomFace.toggle()
                    tracker.set_IsZoomFace(bool(isZoomFace))
                    
            except ValueError as e:
                if "Input timestamp must be monotonically increasing" in str(e):
                    # Reset timestamp if we get an error
                    print("Timestamp error - resetting time tracking")
                    start_time = time.time()
                    continue
                else:
                    print(f"ValueError: {e}")
            except Exception as e:
                print(f"Unexpected error: {str(e)}")
                # Continue running despite errors
    except KeyboardInterrupt:
        print("\nProgram terminated by user")
    finally:
        if 'cap' in locals() and cap.isOpened():
            cap.release()
        cv2.destroyAllWindows()
        print("Resources released. Program ended.")

if __name__ == "__main__":
    main()

2025-04-29 14:33:06.782 python[14221:3831013] WARNING: AVCaptureDeviceTypeExternal is deprecated for Continuity Cameras. Please use AVCaptureDeviceTypeContinuityCamera and add NSCameraUseContinuityCameraDeviceType to your Info.plist.
I0000 00:00:1745911988.206105 3831013 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.4), renderer: Apple M1 Max
W0000 00:00:1745911988.214675 3831013 face_landmarker_graph.cc:174] Sets FaceBlendshapesGraph acceleration to xnnpack by default.
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1745911988.225768 4446964 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1745911988.238656 4446964 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
I0000 00:00:1745911988.322532 3831013 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.4), rendere

Using device: mps
Face tracking started. Press 'q' to quit.
Controls:
  s: Toggle face element labels
  d: Toggle bounding box
  f: Toggle head pose visualization
  g: Toggle face mask
  e: Toggle value output
  z: Toggle face zoom
ValueError: not enough values to unpack (expected 5, got 2)
ValueError: not enough values to unpack (expected 5, got 2)
ValueError: not enough values to unpack (expected 5, got 2)
ValueError: not enough values to unpack (expected 5, got 2)
ValueError: not enough values to unpack (expected 5, got 2)
ValueError: not enough values to unpack (expected 5, got 2)
ValueError: not enough values to unpack (expected 5, got 2)
ValueError: not enough values to unpack (expected 5, got 2)
ValueError: not enough values to unpack (expected 5, got 2)
ValueError: not enough values to unpack (expected 5, got 2)
ValueError: not enough values to unpack (expected 5, got 2)
ValueError: not enough values to unpack (expected 5, got 2)
ValueError: not enough values to unpack (expecte